# Aerial Imagery Classifier

Sam Blake, started 23 December 2022

In [2]:
%load_ext autoreload
%autoreload 2

%matplotlib notebook
%pylab notebook

import os
import time
import glob
import math
import numpy as np
import cv2
import matplotlib as mpl
from matplotlib import pyplot as plt

from tqdm import tqdm

import numba
from numba import jit, prange, set_num_threads

from PIL import Image

import rasterio
from rasterio.plot import show

from aerial_imagery_classifier import imagery_statistical_binary_classification, \
    show_bgr_image, show_bgra_image, show_greyscale_image, denoise 

plt.ioff()
plt.rcParams["figure.figsize"] = (10, 6)

Populating the interactive namespace from numpy and matplotlib


## De Bortolli Rice

In [52]:
tile_42_crop_1 = cv2.imread('../de_bortolli/DeBortolli/map/spectra/tile_42/crop_1.png')
tile_42_dirt_1 = cv2.imread('../de_bortolli/DeBortolli/map/spectra/tile_42/dirt_1.png')
tile_42_water_1 = cv2.imread('../de_bortolli/DeBortolli/map/spectra/tile_42/water_1.png')
tile_42_weed_1 = cv2.imread('../de_bortolli/DeBortolli/map/spectra/tile_42/weed_1.png')

In [53]:
tile_47_crop_1 = cv2.imread('../de_bortolli/DeBortolli/map/spectra/tile_47/crop_1.png')
tile_47_crop_2 = cv2.imread('../de_bortolli/DeBortolli/map/spectra/tile_47/crop_2.png')
tile_47_crop_3 = cv2.imread('../de_bortolli/DeBortolli/map/spectra/tile_47/crop_3.png')
tile_47_dead_1 = cv2.imread('../de_bortolli/DeBortolli/map/spectra/tile_47/dead_1.png')
tile_47_weed_1 = cv2.imread('../de_bortolli/DeBortolli/map/spectra/tile_47/weed_1.png')
tile_47_weed_2 = cv2.imread('../de_bortolli/DeBortolli/map/spectra/tile_47/weed_2.png')

In [59]:
filename = '../de_bortolli/DeBortolli/map/tiles/result_tile_42.tif'
filename = '../de_bortolli/DeBortolli/map/tiles/result_tile_47.tif'
# filename = '../de_bortolli/DeBortolli/map/tiles/result_tile_52.tif'

In [ ]:
dataset = rasterio.open(filename)

red,green,blue = dataset.read(1), dataset.read(2), dataset.read(3)
img = np.zeros((blue.shape[0], blue.shape[1], 3), dtype = np.uint8)
img[:,:,0] = blue
img[:,:,1] = green
img[:,:,2] = red

density, classification_image, classification_confidence, spray_region = \
    imagery_statistical_binary_classification(\
        img, \
        [tile_47_weed_1, tile_47_weed_2, tile_42_weed_1], \
        [tile_47_crop_1, tile_47_crop_2, tile_47_crop_3, tile_47_dead_1, \
            tile_42_dirt_1, tile_42_water_1], \
        image_resize_ratio = 5, \
        image_blur_kernel_size = (7,7), \
        template_blur_kernel_size = (3,3), \
        template_resize_size = (64,64), \
        duplicate_tolerance = 1, \
        common_tolerance = 1, \
        n_sigma_thresholds = [2.75, 3], \
        equalise = False, \
        denoise_classification_map = True, morph_open = 175, morph_close = 10, \
        dilation_size = 301, \
        export_histogram = True, \
        export_heatmap = True, \
        half_normal_dist = True, \
        imagery_id = filename, \
        plotting = True, \
        verbose = True)


# Export spray map to GeoTIFF.
output_filename = filename.replace('.tif',f'_SPRAY_MAP.tif')

kwargs = dataset.meta
kwargs.update(
    dtype=rasterio.uint8,
    count=1,
    compress='lzw')

with rasterio.open(output_filename, 'w', **kwargs) as dst:
    dst.write_band(1, 255*spray_region)

# Export weed map to GeoTIFF.
output_filename = filename.replace('.tif',f'_WEED_MAP.tif')

kwargs = dataset.meta
kwargs.update(
    dtype=rasterio.uint8,
    count=1,
    compress='lzw')

with rasterio.open(output_filename, 'w', **kwargs) as dst:
    dst.write_band(1, np.where(classification_confidence == 0, 0, 255))

# Export classification CI to GeoTIFF.
output_filename = filename.replace('.tif',f'_CLASSIFICATION_CONFIDENCE.tif')

kwargs = dataset.meta
kwargs.update(
    dtype=rasterio.uint8,
    count=1,
    compress='lzw')

with rasterio.open(output_filename, 'w', **kwargs) as dst:
    dst.write_band(1, classification_confidence)

In [58]:
file_pattern = '/Users/user/Documents/Research/rotavision/de_bortolli/DeBortolli/map/tiles/*.tif'
for filename in glob.glob(file_pattern):
    print(os.path.basename(filename))
    
    if 'CLASSIFICATION_MAP' in filename:
        continue
    
    dataset = rasterio.open(filename)

    red,green,blue = dataset.read(1), dataset.read(2), dataset.read(3)
    img = np.zeros((blue.shape[0], blue.shape[1], 3), dtype = np.uint8)
    img[:,:,0] = blue
    img[:,:,1] = green
    img[:,:,2] = red

    density, classification_image, classification_confidence, spray_region = \
        imagery_statistical_binary_classification(\
            img, \
            [tile_47_weed_1, tile_47_weed_2, tile_42_weed_1], \
            [tile_47_crop_1, tile_47_crop_2, tile_47_crop_3, tile_47_dead_1, \
                tile_42_dirt_1, tile_42_water_1], \
            image_resize_ratio = 5, \
            image_blur_kernel_size = (7,7), \
            template_blur_kernel_size = (3,3), \
            template_resize_size = (64,64), \
            duplicate_tolerance = 1, \
            common_tolerance = 1, \
            n_sigma_thresholds = [2.75, 3], \
            equalise = False, \
            denoise_classification_map = True, morph_open = 175, morph_close = 10, \
            dilation_size = 301, \
            half_normal_dist = True, \
            export_histogram = False, \
            export_heatmap = False, \
            imagery_id = filename, \
            plotting = False, \
            verbose = True)


    # Export spray map to GeoTIFF.
    output_filename = filename.replace('.tif',f'_SPRAY_MAP.tif')

    kwargs = dataset.meta
    kwargs.update(
        dtype=rasterio.uint8,
        count=1,
        compress='lzw')

    with rasterio.open(output_filename, 'w', **kwargs) as dst:
        dst.write_band(1, 255*spray_region)

    # Export weed map to GeoTIFF.
    output_filename = filename.replace('.tif',f'_WEED_MAP.tif')

    kwargs = dataset.meta
    kwargs.update(
        dtype=rasterio.uint8,
        count=1,
        compress='lzw')

    with rasterio.open(output_filename, 'w', **kwargs) as dst:
        dst.write_band(1, np.where(classification_confidence == 0, 0, 255))

    # Export classification CI to GeoTIFF.
    output_filename = filename.replace('.tif',f'_CLASSIFICATION_CONFIDENCE.tif')

    kwargs = dataset.meta
    kwargs.update(
        dtype=rasterio.uint8,
        count=1,
        compress='lzw')

    with rasterio.open(output_filename, 'w', **kwargs) as dst:
        dst.write_band(1, classification_confidence)

result_tile_34.tif
number of distinct spectra in sample A is 2645
number of distinct spectra in sample B is 6540
mean = 0.0000, std = 0.0009
   31762 [px] in 2.75 CI, 0.0411 [%] of all pixels. 0.0411 [%] cumulative total.
   15995 [px] in 3.00 CI, 0.0207 [%] of all pixels. 0.0618 [%] cumulative total.
Sample not detected in 99.94 [%] of all pixels.
Spray region is 2.42 [%] of total region.
result_tile_20.tif
number of distinct spectra in sample A is 2645
number of distinct spectra in sample B is 6540
mean = 0.0000, std = 0.0015
   61434 [px] in 2.75 CI, 0.0795 [%] of all pixels. 0.0795 [%] cumulative total.
   27073 [px] in 3.00 CI, 0.0350 [%] of all pixels. 0.1146 [%] cumulative total.
Sample not detected in 99.89 [%] of all pixels.
Spray region is 4.22 [%] of total region.
result_tile_21.tif
number of distinct spectra in sample A is 2645
number of distinct spectra in sample B is 6540
mean = 0.0000, std = 0.0014
    5592 [px] in 2.75 CI, 0.0072 [%] of all pixels. 0.0072 [%] cumulative

Spray region is 2.47 [%] of total region.
result_tile_56.tif
number of distinct spectra in sample A is 2645
number of distinct spectra in sample B is 6540
mean = 0.0000, std = 0.0010
   38280 [px] in 2.75 CI, 0.0496 [%] of all pixels. 0.0496 [%] cumulative total.
   12385 [px] in 3.00 CI, 0.0160 [%] of all pixels. 0.0656 [%] cumulative total.
Sample not detected in 99.93 [%] of all pixels.
Spray region is 2.82 [%] of total region.
result_tile_40.tif
number of distinct spectra in sample A is 2645
number of distinct spectra in sample B is 6540
mean = 0.0000, std = 0.0017
  301171 [px] in 2.75 CI, 0.3899 [%] of all pixels. 0.3899 [%] cumulative total.
  119555 [px] in 3.00 CI, 0.1548 [%] of all pixels. 0.5446 [%] cumulative total.
Sample not detected in 99.46 [%] of all pixels.
Spray region is 13.38 [%] of total region.
result_tile_54.tif
number of distinct spectra in sample A is 2645
number of distinct spectra in sample B is 6540
mean = 0.0000, std = nan
       0 [px] in 2.75 CI, 0.0000 

Spray region is 2.57 [%] of total region.
result_tile_6.tif
number of distinct spectra in sample A is 2645
number of distinct spectra in sample B is 6540
mean = 0.0000, std = 0.0014
  146194 [px] in 2.75 CI, 0.1892 [%] of all pixels. 0.1892 [%] cumulative total.
  119483 [px] in 3.00 CI, 0.1547 [%] of all pixels. 0.3439 [%] cumulative total.
Sample not detected in 99.66 [%] of all pixels.
Spray region is 8.24 [%] of total region.
result_tile_61.tif
number of distinct spectra in sample A is 2645
number of distinct spectra in sample B is 6540
mean = 0.0000, std = 0.0010
   34692 [px] in 2.75 CI, 0.0449 [%] of all pixels. 0.0449 [%] cumulative total.
   12151 [px] in 3.00 CI, 0.0157 [%] of all pixels. 0.0606 [%] cumulative total.
Sample not detected in 99.94 [%] of all pixels.
Spray region is 1.65 [%] of total region.
result_tile_49.tif
number of distinct spectra in sample A is 2645
number of distinct spectra in sample B is 6540
mean = 0.0000, std = 0.0016
  119302 [px] in 2.75 CI, 0.1544

Spray region is 0.00 [%] of total region.
result_tile_14.tif
number of distinct spectra in sample A is 2645
number of distinct spectra in sample B is 6540
mean = 0.0000, std = 0.0008
   91240 [px] in 2.75 CI, 0.1181 [%] of all pixels. 0.1181 [%] cumulative total.
   50856 [px] in 3.00 CI, 0.0658 [%] of all pixels. 0.1839 [%] cumulative total.
Sample not detected in 99.82 [%] of all pixels.
Spray region is 7.56 [%] of total region.
result_tile_16.tif
number of distinct spectra in sample A is 2645
number of distinct spectra in sample B is 6540
mean = 0.0000, std = 0.0011
  126774 [px] in 2.75 CI, 0.1641 [%] of all pixels. 0.1641 [%] cumulative total.
   53874 [px] in 3.00 CI, 0.0697 [%] of all pixels. 0.2338 [%] cumulative total.
Sample not detected in 99.77 [%] of all pixels.
Spray region is 12.12 [%] of total region.
result_tile_17.tif
number of distinct spectra in sample A is 2645
number of distinct spectra in sample B is 6540
mean = 0.0000, std = 0.0016
  109605 [px] in 2.75 CI, 0.14

## TavistockAG Precision Weed Trial

In [31]:
dry_crop_1 = cv2.imread('../TavistockAG_Precision_Weed_Trial/spectra/crop_1.jpg')
dry_crop_2 = cv2.imread('../TavistockAG_Precision_Weed_Trial/spectra/crop_2.jpg')
dry_crop_3 = cv2.imread('../TavistockAG_Precision_Weed_Trial/spectra/crop_3.jpg')
dry_crop_4 = cv2.imread('../TavistockAG_Precision_Weed_Trial/spectra/crop_4.jpg')
dry_crop_5 = cv2.imread('../TavistockAG_Precision_Weed_Trial/spectra/crop_5.jpg')
dry_crop_6 = cv2.imread('../TavistockAG_Precision_Weed_Trial/spectra/crop_6.jpg')
dry_crop_7 = cv2.imread('../TavistockAG_Precision_Weed_Trial/spectra/crop_7.jpg')

dry_crop_8 = cv2.imread('../TavistockAG_Precision_Weed_Trial/spectra/crop_8.jpg')
rocks_1 = cv2.imread('../TavistockAG_Precision_Weed_Trial/spectra/rocks_1.jpg')
rocks_2 = cv2.imread('../TavistockAG_Precision_Weed_Trial/spectra/rocks_2.jpg')
rocks_3 = cv2.imread('../TavistockAG_Precision_Weed_Trial/spectra/rocks_3.jpg')
rocks_4 = cv2.imread('../TavistockAG_Precision_Weed_Trial/spectra/rocks_4.jpg')
rocks_5 = cv2.imread('../TavistockAG_Precision_Weed_Trial/spectra/rocks_5.jpg')

In [32]:
weed_1 = cv2.imread('../TavistockAG_Precision_Weed_Trial/spectra/weed_1.jpg')
weed_2 = cv2.imread('../TavistockAG_Precision_Weed_Trial/spectra/weed_2.jpg')
weed_3 = cv2.imread('../TavistockAG_Precision_Weed_Trial/spectra/weed_3.jpg')

In [138]:
# filename = '../TavistockAG_Precision_Weed_Trial/calibration/100_Orthomosaic_rgb_tile_399.tif'
# filename = '../TavistockAG_Precision_Weed_Trial/calibration/100_Orthomosaic_rgb_tile_446.tif'
# filename = '../TavistockAG_Precision_Weed_Trial/calibration/100_Orthomosaic_rgb_tile_658.tif'
# filename = '../TavistockAG_Precision_Weed_Trial/calibration/100_Orthomosaic_rgb_tile_383.tif'
# filename = '../TavistockAG_Precision_Weed_Trial/calibration/100_Orthomosaic_rgb_tile_607.tif'
filename = '../TavistockAG_Precision_Weed_Trial/calibration/100_Orthomosaic_rgb_tile_1150.tif'
# filename = '../TavistockAG_Precision_Weed_Trial/calibration/100_Orthomosaic_rgb_tile_1151.tif'
# filename = '../TavistockAG_Precision_Weed_Trial/calibration/100_Orthomosaic_rgb_tile_1141.tif'
# filename = '../TavistockAG_Precision_Weed_Trial/calibration/100_Orthomosaic_rgb_tile_1136.tif'
# filename = '../TavistockAG_Precision_Weed_Trial/calibration/100_Orthomosaic_rgb_tile_1137.tif'

In [ ]:
dataset = rasterio.open(filename)

red,green,blue = dataset.read(1), dataset.read(2), dataset.read(3)
img = np.zeros((blue.shape[0], blue.shape[1], 3), dtype = np.uint8)
img[:,:,0] = blue
img[:,:,1] = green
img[:,:,2] = red

density, classification_image, classification_confidence, spray_region = \
    imagery_statistical_binary_classification(\
        img, \
        [weed_1, weed_2, weed_3], \
        [dry_crop_1, dry_crop_2, dry_crop_3, dry_crop_4, dry_crop_5, dry_crop_6, dry_crop_7], \
        image_resize_ratio = 5, \
        image_blur_kernel_size = (7,7), \
        template_blur_kernel_size = (3,3), \
        template_resize_size = (64,64), \
        duplicate_tolerance = 1, \
        common_tolerance = 1, \
        n_sigma_thresholds = [2.5, 2.6, 2.7, 2.8, 2.9, 3], \
        equalise = False, \
        denoise_classification_map = True, morph_open = 175, morph_close = 10, \
        dilation_size = 301, \
        export_histogram = True, \
        export_heatmap = True, \
        half_normal_dist = True, \
        imagery_id = filename, \
        plotting = True, \
        verbose = True)


# Export spray map to GeoTIFF.
output_filename = filename.replace('.tif',f'_SPRAY_MAP.tif')

kwargs = dataset.meta
kwargs.update(
    dtype=rasterio.uint8,
    count=1,
    compress='lzw')

with rasterio.open(output_filename, 'w', **kwargs) as dst:
    dst.write_band(1, 255*spray_region)

# Export weed map to GeoTIFF.
output_filename = filename.replace('.tif',f'_WEED_MAP.tif')

kwargs = dataset.meta
kwargs.update(
    dtype=rasterio.uint8,
    count=1,
    compress='lzw')

with rasterio.open(output_filename, 'w', **kwargs) as dst:
    dst.write_band(1, np.where(classification_confidence == 0, 0, 255))

# Export classification CI to GeoTIFF.
output_filename = filename.replace('.tif',f'_CLASSIFICATION_CONFIDENCE.tif')

kwargs = dataset.meta
kwargs.update(
    dtype=rasterio.uint8,
    count=1,
    compress='lzw')

with rasterio.open(output_filename, 'w', **kwargs) as dst:
    dst.write_band(1, classification_confidence)

In [82]:
# TODO: export json or yaml file of input parameterisations.

In [ ]:
file_pattern = '/Users/user/Documents/Research/rotavision/TavistockAG_Precision_Weed_Trial/tiles/*.tif'
for filename in glob.glob(file_pattern):
    print(os.path.basename(filename))
    
    if 'CLASSIFICATION_MAP' in filename:
        continue
    
    dataset = rasterio.open(filename)

    red,green,blue = dataset.read(1), dataset.read(2), dataset.read(3)
    img = np.zeros((blue.shape[0], blue.shape[1], 3), dtype = np.uint8)
    img[:,:,0] = blue
    img[:,:,1] = green
    img[:,:,2] = red

    density, classification_image, classification_confidence, spray_region = \
        imagery_statistical_binary_classification(\
            img, \
            [weed_1, weed_2, weed_3], \
            [dry_crop_1, dry_crop_2, dry_crop_3, dry_crop_4, dry_crop_5, dry_crop_6, dry_crop_7], \
            image_resize_ratio = 5, \
            image_blur_kernel_size = (7,7), \
            template_blur_kernel_size = (3,3), \
            template_resize_size = (64,64), \
            duplicate_tolerance = 1, \
            common_tolerance = 1, \
            n_sigma_thresholds = [2.5, 2.75, 3], \
            equalise = False, \
            denoise_classification_map = True, morph_open = 175, morph_close = 10, \
            dilation_size = 301, \
            export_histogram = False, \
            export_heatmap = False, \
            half_normal_dist = True, \
            imagery_id = filename, \
            plotting = False, \
            verbose = True)


    # Export spray map to GeoTIFF.
    output_filename = filename.replace('.tif',f'_SPRAY_MAP.tif')

    kwargs = dataset.meta
    kwargs.update(
        dtype=rasterio.uint8,
        count=1,
        compress='lzw')

    with rasterio.open(output_filename, 'w', **kwargs) as dst:
        dst.write_band(1, spray_region)

    # Export weed map to GeoTIFF.
    output_filename = filename.replace('.tif',f'_WEED_MAP.tif')

    kwargs = dataset.meta
    kwargs.update(
        dtype=rasterio.uint8,
        count=1,
        compress='lzw')

    with rasterio.open(output_filename, 'w', **kwargs) as dst:
        dst.write_band(1, np.where(classification_confidence == 0, 0, 255))

    # Export classification CI to GeoTIFF.
    output_filename = filename.replace('.tif',f'_CLASSIFICATION_CONFIDENCE.tif')

    kwargs = dataset.meta
    kwargs.update(
        dtype=rasterio.uint8,
        count=1,
        compress='lzw')

    with rasterio.open(output_filename, 'w', **kwargs) as dst:
        dst.write_band(1, classification_confidence)

## nihill radish

Import features for detection: 

In [12]:
nihill_radish_sample_1 = cv2.imread('../nihill_radish/spectra/nihill_radish_sample_1.jpg')
nihill_radish_sample_2 = cv2.imread('../nihill_radish/spectra/nihill_radish_sample_2.jpg')
nihill_radish_sample_3 = cv2.imread('../nihill_radish/spectra/nihill_radish_sample_3.jpg')
nihill_radish_sample_4 = cv2.imread('../nihill_radish/spectra/nihill_radish_sample_4.jpg')
nihill_radish_sample_5 = cv2.imread('../nihill_radish/spectra/nihill_radish_sample_5.jpg')
nihill_radish_sample_6 = cv2.imread('../nihill_radish/spectra/nihill_radish_sample_6.jpg')
nihill_radish_sample_7 = cv2.imread('../nihill_radish/spectra/nihill_radish_sample_7.jpg')

nihill_radish_sample_1.shape,\
nihill_radish_sample_2.shape,\
nihill_radish_sample_3.shape,\
nihill_radish_sample_4.shape,\
nihill_radish_sample_5.shape,\
nihill_radish_sample_6.shape,\
nihill_radish_sample_7.shape

((243, 552, 3),
 (68, 69, 3),
 (45, 69, 3),
 (62, 29, 3),
 (81, 59, 3),
 (109, 95, 3),
 (105, 129, 3))

In [8]:
wheat_sample_1 = cv2.imread('../nihill_radish/spectra/nihill_crop_sample_1.jpg')
wheat_sample_2 = cv2.imread('../nihill_radish/spectra/nihill_crop_sample_2.jpg')
wheat_sample_3 = cv2.imread('../nihill_radish/spectra/nihill_crop_sample_3.jpg')
wheat_sample_4 = cv2.imread('../nihill_radish/spectra/nihill_crop_sample_4.jpg')
wheat_sample_1.shape,\
wheat_sample_2.shape,\
wheat_sample_3.shape,\
wheat_sample_4.shape

((413, 942, 3), (205, 796, 3), (547, 679, 3), (963, 1149, 3))

In [10]:
wheat_dead_sample_1 = cv2.imread('../nihill_radish/spectra/nihill_crop_dead_sample_1.jpg')
wheat_dead_sample_2 = cv2.imread('../nihill_radish/spectra/nihill_crop_dead_sample_2.jpg')
wheat_dead_sample_3 = cv2.imread('../nihill_radish/spectra/nihill_crop_dead_sample_3.jpg')
wheat_dead_sample_1.shape,\
wheat_dead_sample_2.shape,\
wheat_dead_sample_3.shape

((575, 985, 3), (563, 619, 3), (572, 2050, 3))

In [ ]:
wheat_dead_sample_1, wheat_dead_sample_2, wheat_dead_sample_3

In [ ]:
filename = '/Users/user/Documents/Research/rotavision/nihill_radish/tiles/White_Radish_Orthomosaic_tile_9.tif'
filename = '/Users/user/Documents/Research/rotavision/nihill_radish/tiles/White_Radish_Orthomosaic_tile_47.tif'
filename = '/Users/user/Documents/Research/rotavision/nihill_radish/tiles/White_Radish_Orthomosaic_tile_26.tif'
filename = '/Users/user/Documents/Research/rotavision/nihill_radish/tiles/White_Radish_Orthomosaic_tile_24.tif'

dataset = rasterio.open(filename)

red,green,blue = dataset.read(1), dataset.read(2), dataset.read(3)
img = np.zeros((blue.shape[0], blue.shape[1], 3), dtype = np.uint8)
img[:,:,0] = blue
img[:,:,1] = green
img[:,:,2] = red

density, classification_image, classification_confidence, spray_region = \
    imagery_statistical_binary_classification(\
        img, \
        [nihill_radish_sample_1], \
        [wheat_sample_1, wheat_sample_2, wheat_sample_3, wheat_sample_4], \
        image_resize_ratio = 10, \
        image_blur_kernel_size = (19,19), \
        template_blur_kernel_size = (7,7), \
        template_resize_size = (64,64), \
        duplicate_tolerance = 5, \
        common_tolerance = 1, \
        n_sigma_thresholds = [2.75, 3], \
        equalise = True, \
        denoise_classification_map = True, morph_open = 175, morph_close = 20, \
        dilation_size = 100, \
        limit_outliers = False, \
        half_normal_dist = True, \
        export_histogram = False, \
        export_heatmap = False, \
        imagery_id = filename, \
        plotting = True, \
        verbose = True)

In [ ]:
file_pattern = '/Users/user/Documents/Research/rotavision/nihill_radish/tiles/*.tif'
for filename in glob.glob(file_pattern):
    print(os.path.basename(filename))
    
    if 'CLASSIFICATION_MAP' in filename:
        continue
    
    dataset = rasterio.open(filename)

    red,green,blue = dataset.read(1), dataset.read(2), dataset.read(3)
    img = np.zeros((blue.shape[0], blue.shape[1], 3), dtype = np.uint8)
    img[:,:,0] = blue
    img[:,:,1] = green
    img[:,:,2] = red

    density, classification_image, classification_confidence, spray_region = \
        imagery_statistical_binary_classification(\
            img, \
            [nihill_radish_sample_1], \
            [wheat_sample_1, wheat_sample_2, wheat_sample_3, wheat_sample_4], \
            image_resize_ratio = 10, \
            image_blur_kernel_size = (19,19), \
            template_blur_kernel_size = (7,7), \
            template_resize_size = (64,64), \
            duplicate_tolerance = 5, \
            common_tolerance = 1, \
            n_sigma_thresholds = [2.75, 3], \
            equalise = True, \
            denoise_classification_map = True, morph_open = 175, morph_close = 20, \
            dilation_size = 100, \
            limit_outliers = False, \
            half_normal_dist = True, \
            export_histogram = False, \
            export_heatmap = False, \
            imagery_id = filename, \
            plotting = False, \
            verbose = True)


    # Export spray map to GeoTIFF.
    output_filename = filename.replace('.tif',f'_SPRAY_MAP.tif')

    kwargs = dataset.meta
    kwargs.update(
        dtype=rasterio.uint8,
        count=1,
        compress='lzw')

    with rasterio.open(output_filename, 'w', **kwargs) as dst:
        dst.write_band(1, 255*spray_region)

    # Export weed map to GeoTIFF.
    output_filename = filename.replace('.tif',f'_WEED_MAP.tif')

    kwargs = dataset.meta
    kwargs.update(
        dtype=rasterio.uint8,
        count=1,
        compress='lzw')

    with rasterio.open(output_filename, 'w', **kwargs) as dst:
        dst.write_band(1, np.where(classification_confidence == 0, 0, 255))

    # Export classification CI to GeoTIFF.
    output_filename = filename.replace('.tif',f'_CLASSIFICATION_CONFIDENCE.tif')

    kwargs = dataset.meta
    kwargs.update(
        dtype=rasterio.uint8,
        count=1,
        compress='lzw')

    with rasterio.open(output_filename, 'w', **kwargs) as dst:
        dst.write_band(1, classification_confidence)

White_Radish_Orthomosaic_tile_28.tif
number of distinct spectra in sample A is 259
number of distinct spectra in sample B is 449


/usr/local/lib/python3.9/site-packages/numpy/lib/nanfunctions.py:1670: RuntimeWarning: Degrees of freedom <= 0 for slice.
  var = nanvar(a, axis=axis, dtype=dtype, out=out, ddof=ddof,


mean = 0.0000, std = nan
       0 [px] in 2.75 CI, 0.0000 [%] of all pixels. 0.0000 [%] cumulative total.
       0 [px] in 3.00 CI, 0.0000 [%] of all pixels. 0.0000 [%] cumulative total.
Sample not detected in 100.00 [%] of all pixels.
Spray region is 0.00 [%] of total region.
White_Radish_Orthomosaic_tile_14.tif
number of distinct spectra in sample A is 259
number of distinct spectra in sample B is 449
mean = 0.0000, std = nan
       0 [px] in 2.75 CI, 0.0000 [%] of all pixels. 0.0000 [%] cumulative total.
       0 [px] in 3.00 CI, 0.0000 [%] of all pixels. 0.0000 [%] cumulative total.
Sample not detected in 100.00 [%] of all pixels.
Spray region is 0.00 [%] of total region.
White_Radish_Orthomosaic_tile_15.tif
number of distinct spectra in sample A is 259
number of distinct spectra in sample B is 449
mean = 0.0000, std = 0.0019
  244390 [px] in 2.75 CI, 0.5731 [%] of all pixels. 0.5731 [%] cumulative total.
  200440 [px] in 3.00 CI, 0.4700 [%] of all pixels. 1.0431 [%] cumulative tot